# Classification in Spark

The intent of this blog is to demonstrate binary classification in pySpark. The various steps involved in developing a classification model in pySpark are as follows:

1) Initialize a Spark session

2) Download and read the the dataset

3) Developing initial understanding about the data

4) Handling missing values

5) Scalerizing the features

6) Train test split

7) Imbalance handling

8) Feature selection

9) Performance evaluation

In [1]:
# Initializing a Spark session
from pyspark.sql import SparkSession
import time
spark = SparkSession.builder.master("local").appName("poker").config("spark.some.config.option","some-value").getOrCreate()

# Download and read the dataset

In [2]:
!pwd
!ls

/home/hadoop
diabetes.csv			    logisticregression-flight.ipynb
hw3.ipynb			    logisticregression-Income.ipynb
launchJupyter.sh		    logisticregression-poker.ipynb
logisticregression_diabeties.ipynb  spark_2.ipynb


In [3]:
# create a Spark dataframe named 'raw_data'
start_time = time.time()
raw_data = spark.read.csv('s3://516ml/poker_hand.csv',
                    header='true', inferSchema='true')
print("--- %s seconds ---" % (time.time() - start_time))

raw_data.columns

--- 13.2866048813 seconds ---


['s1', 'c1', 's2', 'c2', 's3', 'c3', 's4', 'c4', 's5', 'c5', 'Class']

In [4]:
raw_data.show(5)

+---+---+---+---+---+---+---+---+---+---+-----+
| s1| c1| s2| c2| s3| c3| s4| c4| s5| c5|Class|
+---+---+---+---+---+---+---+---+---+---+-----+
|  4|  7|  3|  5|  3|  3|  1| 13|  4|  8|    0|
|  2|  8|  4|  9|  4|  6|  4|  1|  3|  7|    0|
|  3|  6|  1|  3|  2| 11|  3|  9|  2|  3|    1|
|  2| 10|  2|  5|  4| 13|  3|  9|  1|  6|    0|
|  3|  2|  1|  3|  4|  7|  3|  5|  1| 11|    0|
+---+---+---+---+---+---+---+---+---+---+-----+
only showing top 5 rows



In [5]:
raw_data.describe().select("s1","c1","s2", "c2", "s3", "c3", "s4", "c4", "s5", "c5", "Class").show()

+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|               s1|                c1|               s2|               c2|                s3|                c3|                s4|                c4|               s5|                c5|             Class|
+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+
|           800000|            800000|           800000|           800000|            800000|            800000|            800000|            800000|           800000|            800000|            800000|
|        2.5004525|          7.003275|       2.49920375|       7.00594375|         2.4987175|         6.9897175|        2.49958375|         6.9983725|       2.50062125|    

replace all the zeros in the abaove mentioned fields (except "Pregnancies") with NaN.

In [6]:
import numpy as np
from pyspark.sql.functions import when
raw_data=raw_data.withColumn("Class",when(raw_data.Class == 0,0.0).otherwise(1.0))
cols = ["s1","c1","s2", "c2", "s3", "c3", "s4", "c4", "s5", "c5", "Class"]

from pyspark.sql import functions as F

for col in raw_data.columns:
     raw_data= raw_data.withColumn(col,F.col(col).cast("float"))

In [9]:
raw_data.show()

+---+----+---+----+---+----+---+----+---+----+-----+
| s1|  c1| s2|  c2| s3|  c3| s4|  c4| s5|  c5|Class|
+---+----+---+----+---+----+---+----+---+----+-----+
|4.0| 7.0|3.0| 5.0|3.0| 3.0|1.0|13.0|4.0| 8.0|  0.0|
|2.0| 8.0|4.0| 9.0|4.0| 6.0|4.0| 1.0|3.0| 7.0|  0.0|
|3.0| 6.0|1.0| 3.0|2.0|11.0|3.0| 9.0|2.0| 3.0|  1.0|
|2.0|10.0|2.0| 5.0|4.0|13.0|3.0| 9.0|1.0| 6.0|  0.0|
|3.0| 2.0|1.0| 3.0|4.0| 7.0|3.0| 5.0|1.0|11.0|  0.0|
|2.0| 1.0|3.0| 6.0|2.0|10.0|4.0|11.0|2.0| 6.0|  1.0|
|2.0| 3.0|2.0| 4.0|3.0| 5.0|4.0|12.0|1.0| 6.0|  0.0|
|4.0| 9.0|4.0| 7.0|3.0| 7.0|1.0| 6.0|2.0| 4.0|  1.0|
|4.0| 7.0|2.0| 4.0|3.0| 7.0|2.0| 8.0|2.0|11.0|  1.0|
|4.0| 8.0|3.0| 8.0|1.0|11.0|3.0| 5.0|2.0| 1.0|  1.0|
|1.0| 7.0|3.0|10.0|3.0| 7.0|3.0| 4.0|2.0| 1.0|  1.0|
|1.0| 3.0|4.0| 2.0|3.0|11.0|2.0| 2.0|3.0| 4.0|  1.0|
|4.0| 1.0|2.0| 9.0|4.0| 6.0|3.0| 2.0|1.0| 3.0|  0.0|
|2.0| 2.0|1.0| 2.0|4.0| 8.0|4.0|12.0|3.0| 4.0|  1.0|
|4.0|11.0|1.0| 5.0|3.0| 6.0|2.0|13.0|4.0| 7.0|  0.0|
|1.0| 9.0|2.0| 8.0|4.0|11.0|4.0| 9.0|1.0|11.0|

So we have replaced all "0" with NaN. Now, we can simply impute the NaN by calling an imputer :)

In [10]:
from pyspark.ml.feature import Imputer
cols = ["s1","c1","s2", "c2", "s3", "c3", "s4", "c4", "s5", "c5", "Class"]

imputer=Imputer(inputCols=cols,outputCols=cols)
model=imputer.fit(raw_data)
raw_data=model.transform(raw_data)
raw_data.show(5)

+---+----+---+---+---+----+---+----+---+----+-----+
| s1|  c1| s2| c2| s3|  c3| s4|  c4| s5|  c5|Class|
+---+----+---+---+---+----+---+----+---+----+-----+
|4.0| 7.0|3.0|5.0|3.0| 3.0|1.0|13.0|4.0| 8.0|  0.0|
|2.0| 8.0|4.0|9.0|4.0| 6.0|4.0| 1.0|3.0| 7.0|  0.0|
|3.0| 6.0|1.0|3.0|2.0|11.0|3.0| 9.0|2.0| 3.0|  1.0|
|2.0|10.0|2.0|5.0|4.0|13.0|3.0| 9.0|1.0| 6.0|  0.0|
|3.0| 2.0|1.0|3.0|4.0| 7.0|3.0| 5.0|1.0|11.0|  0.0|
+---+----+---+---+---+----+---+----+---+----+-----+
only showing top 5 rows



combine all the features in one single feature vector. 

In [11]:
cols.remove("Class")
# import the vector assembler
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=cols,outputCol="features")
# use the transform method to transform our dataset
raw_data=assembler.transform(raw_data)
raw_data.select("features").show(truncate=False)

+-------------------------------------------+
|features                                   |
+-------------------------------------------+
|[4.0,7.0,3.0,5.0,3.0,3.0,1.0,13.0,4.0,8.0] |
|[2.0,8.0,4.0,9.0,4.0,6.0,4.0,1.0,3.0,7.0]  |
|[3.0,6.0,1.0,3.0,2.0,11.0,3.0,9.0,2.0,3.0] |
|[2.0,10.0,2.0,5.0,4.0,13.0,3.0,9.0,1.0,6.0]|
|[3.0,2.0,1.0,3.0,4.0,7.0,3.0,5.0,1.0,11.0] |
|[2.0,1.0,3.0,6.0,2.0,10.0,4.0,11.0,2.0,6.0]|
|[2.0,3.0,2.0,4.0,3.0,5.0,4.0,12.0,1.0,6.0] |
|[4.0,9.0,4.0,7.0,3.0,7.0,1.0,6.0,2.0,4.0]  |
|[4.0,7.0,2.0,4.0,3.0,7.0,2.0,8.0,2.0,11.0] |
|[4.0,8.0,3.0,8.0,1.0,11.0,3.0,5.0,2.0,1.0] |
|[1.0,7.0,3.0,10.0,3.0,7.0,3.0,4.0,2.0,1.0] |
|[1.0,3.0,4.0,2.0,3.0,11.0,2.0,2.0,3.0,4.0] |
|[4.0,1.0,2.0,9.0,4.0,6.0,3.0,2.0,1.0,3.0]  |
|[2.0,2.0,1.0,2.0,4.0,8.0,4.0,12.0,3.0,4.0] |
|[4.0,11.0,1.0,5.0,3.0,6.0,2.0,13.0,4.0,7.0]|
|[1.0,9.0,2.0,8.0,4.0,11.0,4.0,9.0,1.0,11.0]|
|[1.0,4.0,4.0,11.0,2.0,10.0,4.0,1.0,4.0,4.0]|
|[2.0,10.0,2.0,11.0,2.0,3.0,2.0,4.0,1.0,2.0]|
|[4.0,13.0,2.0,1.0,3.0,5.0,1.0,1.0

# Standard Sclarizer 

So we have created a feature vector. Now let us use StandardScaler to scalerize the newly created "feature" column 

In [12]:
from pyspark.ml.feature import StandardScaler
standardscaler=StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
raw_data=standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("features","Scaled_features").show(5)

+--------------------+--------------------+
|            features|     Scaled_features|
+--------------------+--------------------+
|[4.0,7.0,3.0,5.0,...|[3.57929638176889...|
|[2.0,8.0,4.0,9.0,...|[1.78964819088444...|
|[3.0,6.0,1.0,3.0,...|[2.68447228632666...|
|[2.0,10.0,2.0,5.0...|[1.78964819088444...|
|[3.0,2.0,1.0,3.0,...|[2.68447228632666...|
+--------------------+--------------------+
only showing top 5 rows



# Train, test split

Now that the preprocessing of the data is complete. Let us split the dataset in training and testing set. 

In [13]:
train, test = raw_data.randomSplit([0.8, 0.2], seed=12345)

Let us check whether their is imbalance in the dataset

In [14]:
dataset_size=float(train.select("Class").count())
numPositives=train.select("Class").where('Class == 1').count()
per_ones=(float(numPositives)/float(dataset_size))*100
numNegatives=float(dataset_size-numPositives)
print('The number of ones are {}'.format(numPositives))
print('Percentage of ones are {}'.format(per_ones))

The number of ones are 319077
Percentage of ones are 49.8915628552


# Imbalancing handling

Since the percentage of ones in the dataset is just 49.89 % there is no imbalance in the dataset. 

In [15]:
BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

BalancingRatio = 0.501084371448


In [17]:
from pyspark.sql.functions import when
train=train.withColumn("classWeights", when(train.Class == 1,BalancingRatio).otherwise(1-BalancingRatio))
train.select("classWeights").show(5)

+------------------+
|      classWeights|
+------------------+
|0.5010843714476476|
|0.5010843714476476|
|0.5010843714476476|
|0.4989156285523524|
|0.4989156285523524|
+------------------+
only showing top 5 rows



# Feature selection

We use the ChiSqSelector provided by Spark ML for selecting significant features. Please refer my previous blog for more details about working of the ChiSqSelector.   

# Building a classification model using Logistic Regression (LR)

In [18]:
from pyspark.ml.classification import LogisticRegression
start_time = time.time()
lr = LogisticRegression(labelCol="Class", featuresCol="Scaled_features",weightCol="classWeights",maxIter=10)
model = lr.fit(train)    

print("--- %s seconds ---" % (time.time() - start_time))
predict_train=model.transform(train)
predict_test=model.transform(test)

predict_test.select("Class","prediction").show(10)

--- 7.70885491371 seconds ---
+-----+----------+
|Class|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 10 rows



# Evaluating the model

Now let us evaluate the model using BinaryClassificationEvaluator class in Spark ML. BinaryClassificationEvaluator by default uses areaUnderROC as the performance metric 

In [20]:
# The BinaryClassificationEvaluator uses areaUnderROC as the default metric. As of now we will continue with the same
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="Class")

In [21]:
predict_test.select("Class","rawPrediction","prediction","probability").show(5)

+-----+--------------------+----------+--------------------+
|Class|       rawPrediction|prediction|         probability|
+-----+--------------------+----------+--------------------+
|  0.0|[-0.0135468291428...|       1.0|[0.49661334450642...|
|  0.0|[0.00263747611009...|       0.0|[0.50065936864529...|
|  0.0|[3.97057425569571...|       0.0|[0.50009926435508...|
|  0.0|[-0.0044332015137...|       1.0|[0.49889170143671...|
|  0.0|[0.00284999559641...|       0.0|[0.50071249841683...|
+-----+--------------------+----------+--------------------+
only showing top 5 rows



In [22]:
print("The area under ROC for train set is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set is {}".format(evaluator.evaluate(predict_test)))

The area under ROC for train set is 0.502290170117
The area under ROC for test set is 0.500179749646


# Hyper parameters

To this point we have developed a classification model using logistic regression. However, the working of logistic regression depends upon the on a number of parameters. As of now we have worked with only the default parameters. Now, let s try to tune the hyperparameters and see whether it make any difference.  

In [ ]:
# if you are unsure which parameters to tune pls use "print(lr.explainParams())" to get the list of parameters available for tuning  
print(lr.explainParams())

# List of tunable parameters in LR

1) aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)

2) elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)

3) family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)

4) featuresCol: features column name. (default: features, current: Aspect)

5) fitIntercept: whether to fit an intercept term. (default: True)

6) labelCol: label column name. (default: label, current: Outcome)

7) maxIter: max number of iterations (>= 0). (default: 100, current: 10)

8) predictionCol: prediction column name. (default: prediction)

9) probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)

10) rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)

11) regParam: regularization parameter (>= 0). (default: 0.0)

12) standardization: whether to standardize the training features before fitting the model. (default: True)

13) threshold: Threshold in binary classification prediction, in range [0, 1].

14) If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p]. (default: 0.5)

15) thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is the original probability of that class and t is the class's threshold. (undefined)

16) tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)

17) weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (current: classWeights)


Now let us tune some of these parameters and observe their effect on the performance of the algorithm.

For the purpose of hyperparameter tuning we will consider the following parameters:

1) aggregationDepth [2, 5, 10]

2) elasticNetParam [0.0, 0.5, 1.0]

3) fitIntercept [True / False]

4) maxIter [10, 100, 1000]

5) regParam [0.01, 0.5, 2.0]

frist off all let us define a parameter grid as follows:

In [50]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = ParamGridBuilder()\
    .addGrid(lr.aggregationDepth,[2,5,10])\
    .addGrid(lr.elasticNetParam,[0.0, 0.5, 1.0])\
    .addGrid(lr.fitIntercept,[False, True])\
    .addGrid(lr.maxIter,[10, 100, 1000])\
    .addGrid(lr.regParam,[0.01, 0.5, 2.0]) \
    .build()

# https://spark.apache.org/docs/2.1.0/ml-tuning.html

# K-fold cross validation

In [51]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(train)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing
predict_train=cvModel.transform(train)
predict_test=cvModel.transform(test)
print("The area under ROC for train set after CV  is {}".format(evaluator.evaluate(predict_train)))
print("The area under ROC for test set after CV  is {}".format(evaluator.evaluate(predict_test)))

The area under ROC for train set after CV  is 0.502293334574
The area under ROC for test set after CV  is 0.500314937307


In [52]:
print((raw_data.count(), len(raw_data.columns)))

(800000, 13)
